In [ ]:
# Liste des packages nécessaire
packages <- c("keras", "mlbench", "dplyr", "magrittr", "neuralnet", "ggplot2", "reshape2", "tfruns", "randomForest", "tidyverse")

# Fonction pour vérifier et installer les packages
install_missing_packages <- function(packages) {
  for (pkg in packages) {
    if (!requireNamespace(pkg, quietly = TRUE)) {
      message(paste("Le package", pkg, "n'est pas installé. Installation en cours..."))
      install.packages(pkg, dependencies = TRUE)
      #library(pkg)  # Charge le package
    } else {
      message(paste("Le package", pkg, "est déjà installé."))
      #library(pkg)  # Charge le package
    }
  }
}

# Appel de la fonction
install_missing_packages(packages)

library( keras )
library( mlbench )
library( dplyr )
library( magrittr )
library( neuralnet )
library( ggplot2 )
library( reshape2 )
library( tfruns )
library( randomForest )
library( tidyverse )


# ANN

## AB_tot

### Import data

In [ ]:
# Data
VAR_REP = "AB_tot"
test = read.csv2("/content/AB_tot_test.csv")
training = read.csv2("/content/AB_tot_train.csv")

data = rbind(training,test)

dim(data)


[1] 3219   26

### Data preparation

In [ ]:

ind_var_rep <- which(names(training) == VAR_REP)
trainingtarget <- training[, ind_var_rep]
trainingtarget = sqrt(trainingtarget)
# trainingtarget = round(trainingtarget/25)
training = training[,-ind_var_rep]



ind_var_rep <- which(names(test) == VAR_REP)
testtarget <- test[, ind_var_rep]
testtarget = sqrt(testtarget)
# testtarget = round(testtarget/25)
test = test[,-ind_var_rep]



# Matrix
training %<>% mutate_if(is.factor, as.numeric)
training <- as.matrix(training)
dimnames(training) <- NULL

test %<>% mutate_if(is.factor, as.numeric)
test <- as.matrix(test)
dimnames(test) <- NULL


# Normalize
m <- colMeans(training)
s <- apply(training, 2, sd)
training <- scale(training, center = m, scale = s)
test <- scale(test, center = m, scale = s)

### Default model

In [ ]:
# Create Model : ANN_1 ----------------------------------------------------
ANN_1 <- keras_model_sequential()
ANN_1 %>%
  layer_dense(units = 5, activation = 'relu', input_shape = c(25)) %>%
  layer_dense(units = 1)

# Compile
ANN_1 %>% compile(loss = 'mse',
                  optimizer = 'rmsprop',
                  metrics = 'mae')
summary(ANN_1)

# Fit ANN_1
myANN_1 <- ANN_1 %>%
  fit(training,
      trainingtarget,
      epochs = 100,
      #batch_size = 1,
      validation_split = 0.2)
plot(myANN_1)

# Evaluate
ANN_1 %>% evaluate(test, testtarget)
ANN_1_pred <- ANN_1 %>% predict(test)
ANN_1_mse = mean((testtarget-ANN_1_pred)^2) # loss -> mse
ANN_1_mae =mean(abs(ANN_1_pred - testtarget),na.rm=TRUE) # MAE
ANN_1_rmse =sqrt(mean((testtarget - ANN_1_pred)^2,na.rm=TRUE)) # rmse
SSR <- sum((ANN_1_pred - testtarget) ^ 2,na.rm=TRUE)
SST <- sum((testtarget - mean(testtarget)) ^ 2)
ANN_1_rsquare = 1 - (SSR / SST) # r_adj <-
ANN_1_plot_cor = plot(testtarget, ANN_1_pred,main="ANN_1")
ANN_1_cor = cor(testtarget,ANN_1_pred)^2
ANN_1_results = data.frame(model = "ANN_1",mse = ANN_1_mse, mae = ANN_1_mae,
                            rmse = ANN_1_rmse, r_square =ANN_1_rsquare, cor_pred_obs= ANN_1_cor)
ANN_1_results

### Tuning

In [ ]:
# Hyperparameter tuning ---------------------------------------------------

runs <- tuning_run("/content/Experiment.R",
                   flags = list(dense_units1 = c(32, 64),
                                dense_units2 = c(16, 32),
                                dense_units3 = c(8, 16),
                                dense_units4 = c(4, 8),
                                dropout1 = c(0.4, 0.5),
                                dropout1 = c(0.3, 0.4),
                                dropout1 = c(0.2, 0.3),
                                dropout1 = c(0.1, 0.2),
                                batch_size = c(32, 64)))


save_runs = runs
write.csv2(x =save_runs,file = paste0("/content/", VAR_REP, "_runs.csv"), row.names = FALSE)

## BM_tot

### Import data

In [ ]:
# Data
VAR_REP = "BM_tot"
test = read.csv2("/content/BM_tot_test.csv")
training = read.csv2("/content/BM_tot_train.csv")

data = rbind(training,test)

dim(data)

Warning message in file(file, "rt"):
“cannot open file '/content/BM_tot_test.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


### Data preparation

In [ ]:
ind_var_rep <- which(names(training) == VAR_REP)
trainingtarget <- training[, ind_var_rep]
trainingtarget = sqrt(trainingtarget)
# trainingtarget = round(trainingtarget/25)
training = training[,-ind_var_rep]



ind_var_rep <- which(names(test) == VAR_REP)
testtarget <- test[, ind_var_rep]
testtarget = sqrt(testtarget)
# testtarget = round(testtarget/25)
test = test[,-ind_var_rep]



# Matrix
training %<>% mutate_if(is.factor, as.numeric)
training <- as.matrix(training)
dimnames(training) <- NULL

test %<>% mutate_if(is.factor, as.numeric)
test <- as.matrix(test)
dimnames(test) <- NULL


# Normalize
m <- colMeans(training)
s <- apply(training, 2, sd)
training <- scale(training, center = m, scale = s)
test <- scale(test, center = m, scale = s)

### Default model

In [ ]:
# Create Model : ANN_1 ----------------------------------------------------
ANN_1 <- keras_model_sequential()
ANN_1 %>%
  layer_dense(units = 5, activation = 'relu', input_shape = c(25)) %>%
  layer_dense(units = 1)

# Compile
ANN_1 %>% compile(loss = 'mse',
                  optimizer = 'rmsprop',
                  metrics = 'mae')
summary(ANN_1)

# Fit ANN_1
myANN_1 <- ANN_1 %>%
  fit(training,
      trainingtarget,
      epochs = 100,
      #batch_size = 1,
      validation_split = 0.2)
plot(myANN_1)

# Evaluate
ANN_1 %>% evaluate(test, testtarget)
ANN_1_pred <- ANN_1 %>% predict(test)
ANN_1_mse = mean((testtarget-ANN_1_pred)^2) # loss -> mse
ANN_1_mae =mean(abs(ANN_1_pred - testtarget),na.rm=TRUE) # MAE
ANN_1_rmse =sqrt(mean((testtarget - ANN_1_pred)^2,na.rm=TRUE)) # rmse
SSR <- sum((ANN_1_pred - testtarget) ^ 2,na.rm=TRUE)
SST <- sum((testtarget - mean(testtarget)) ^ 2)
ANN_1_rsquare = 1 - (SSR / SST) # r_adj <-
ANN_1_plot_cor = plot(testtarget, ANN_1_pred,main="ANN_1")
ANN_1_cor = cor(testtarget,ANN_1_pred)^2
ANN_1_results = data.frame(model = "ANN_1",mse = ANN_1_mse, mae = ANN_1_mae,
                            rmse = ANN_1_rmse, r_square =ANN_1_rsquare, cor_pred_obs= ANN_1_cor)
ANN_1_results

### Tuning

In [ ]:
VAR_REP

In [ ]:
# Hyperparameter tuning ---------------------------------------------------

runs_bm_tot <- tuning_run("/content/Experiment.R",
                   flags = list(dense_units1 = c(32, 64),
                                dense_units2 = c(16, 32),
                                dense_units3 = c(8, 16),
                                dense_units4 = c(4, 8),
                                dropout1 = c(0.4, 0.5),
                                dropout1 = c(0.3, 0.4),
                                dropout1 = c(0.2, 0.3),
                                dropout1 = c(0.1, 0.2),
                                batch_size = c(32, 64)))


save_runs_bm_tot = runs_bm_tot
write.csv2(x =save_runs_bm_tot,file = paste0("/content/", VAR_REP, "_runs.csv"), row.names = FALSE)

## Richesse_tot

In [ ]:
# Data
VAR_REP = "Richesse_tot"
test = read.csv2("/content/Richesse_tot_test.csv")
training = read.csv2("/content/Richesse_tot_train.csv")

data = rbind(training,test)

dim(data)

[1] 3268   26

### Data preparartion

In [ ]:
ind_var_rep <- which(names(training) == VAR_REP)
trainingtarget <- training[, ind_var_rep]
#trainingtarget = sqrt(trainingtarget)
# trainingtarget = round(trainingtarget/25)
training = training[,-ind_var_rep]



ind_var_rep <- which(names(test) == VAR_REP)
testtarget <- test[, ind_var_rep]
#testtarget = sqrt(testtarget)
# testtarget = round(testtarget/25)
test = test[,-ind_var_rep]



# Matrix
training %<>% mutate_if(is.factor, as.numeric)
training <- as.matrix(training)
dimnames(training) <- NULL

test %<>% mutate_if(is.factor, as.numeric)
test <- as.matrix(test)
dimnames(test) <- NULL


# Normalize
m <- colMeans(training)
s <- apply(training, 2, sd)
training <- scale(training, center = m, scale = s)
test <- scale(test, center = m, scale = s)

### Default model

In [ ]:
# Create Model : ANN_1 ----------------------------------------------------
ANN_1 <- keras_model_sequential()
ANN_1 %>%
  layer_dense(units = 5, activation = 'relu', input_shape = c(25)) %>%
  layer_dense(units = 1)

# Compile
ANN_1 %>% compile(loss = 'mse',
                  optimizer = 'rmsprop',
                  metrics = 'mae')
summary(ANN_1)

# Fit ANN_1
myANN_1 <- ANN_1 %>%
  fit(training,
      trainingtarget,
      epochs = 100,
      #batch_size = 1,
      validation_split = 0.2)
plot(myANN_1)

# Evaluate
ANN_1 %>% evaluate(test, testtarget)
ANN_1_pred <- ANN_1 %>% predict(test)
ANN_1_mse = mean((testtarget-ANN_1_pred)^2) # loss -> mse
ANN_1_mae =mean(abs(ANN_1_pred - testtarget),na.rm=TRUE) # MAE
ANN_1_rmse =sqrt(mean((testtarget - ANN_1_pred)^2,na.rm=TRUE)) # rmse
SSR <- sum((ANN_1_pred - testtarget) ^ 2,na.rm=TRUE)
SST <- sum((testtarget - mean(testtarget)) ^ 2)
ANN_1_rsquare = 1 - (SSR / SST) # r_adj <-
ANN_1_plot_cor = plot(testtarget, ANN_1_pred,main="ANN_1")
ANN_1_cor = cor(testtarget,ANN_1_pred)^2
ANN_1_results = data.frame(model = "ANN_1",mse = ANN_1_mse, mae = ANN_1_mae,
                            rmse = ANN_1_rmse, r_square =ANN_1_rsquare, cor_pred_obs= ANN_1_cor)
ANN_1_results

### Tuning

In [ ]:
VAR_REP

[1] "Richesse_tot"

In [ ]:
# Hyperparameter tuning ---------------------------------------------------

runs_rich_tot <- tuning_run("/content/Experiment.R",
                   flags = list(dense_units1 = c(32, 64),
                                dense_units2 = c(16, 32),
                                dense_units3 = c(8, 16),
                                dense_units4 = c(4, 8),
                                dropout1 = c(0.4, 0.5),
                                dropout1 = c(0.3, 0.4),
                                dropout1 = c(0.2, 0.3),
                                dropout1 = c(0.1, 0.2),
                                batch_size = c(32, 64)))


save_runs_rich_tot = runs_rich_tot
write.csv2(x =save_runs_rich_tot,file = paste0("/content/", VAR_REP, "_runs.csv"), row.names = FALSE)

# RF

In [22]:
ForetAlea <- function(var_rep, df_app, df_valid,mtry,ntree,maxnodes){


  col_posi <- which(names(df_app) == var_rep)
  ForeVDT <- randomForest::randomForest(df_app[-col_posi], df_app[[col_posi]], mtry=mtry,
                          ntree=ntree,maxnodes=maxnodes)

  # Prediction sur le jeu de validation
  col_posi <- which(names(df_valid) == var_rep)
  pred.RF<-predict(ForeVDT,newdata=df_valid[,-col_posi])

  # Calcul du RMSE pour évaluer la qualité du modele
  rmse <- sqrt(mean((df_valid[,col_posi] - pred.RF)^2))

  # calcule du R-squared
  r_adj = cor(df_valid[,col_posi],pred.RF)^2

  # calcule MAE
  MAE <- mean(abs(pred.RF - df_valid[,col_posi]))

  results <- list(RMSE = rmse, R_squared= r_adj,MAE = MAE, model=ForeVDT)


  return(results)
}

# Grille de hyperparametisation
RF_df_grid <- expand.grid(ntree = c(100,300,500,700,900,1000,1300,1500,1700,2000),
                       mtry = c(2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24),
                       nodesize = c(10 , 20,  30,  40,  50,  60,  70,  80))
nrow(RF_df_grid)

[1] 960

## AB_tot

### Import data

In [23]:

# Data
var_rep = "AB_tot"

df_app = read.csv2("/content/AB_tot_train.csv")
df_valid = read.csv2("/content/AB_tot_test.csv")

data = rbind(df_app,df_valid)

dim(data)



[1] 3219   26

### Data preparation

In [24]:
# str(df_app)
# str(df_valid)

ind_var_rep <- which(names(df_app) == var_rep)
trainingtarget <- df_app[, ind_var_rep]
trainingtarget = sqrt(trainingtarget)
training = df_app[,-ind_var_rep]
training[,1:19] = scale(training [,1:19])
df_app = cbind(trainingtarget,training)
colnames(df_app)[ind_var_rep] <- var_rep
df_app = as.data.frame(df_app)

ind_var_rep <- which(names(df_valid) == var_rep)
testtarget <- df_valid[, ind_var_rep]
testtarget <- sqrt(testtarget)
test = df_valid[,-ind_var_rep]
test[,1:19] = scale(test[,1:19])
df_valid = cbind(testtarget,test)
colnames(df_valid)[ind_var_rep] <- var_rep
df_valid = as.data.frame(df_valid)

# str(df_app)
# str(df_valid)

df_app = as.data.frame(df_app)
df_valid = as.data.frame(df_valid)

dim(df_app)
dim(df_valid)

[1] 2353   26

[1] 866  26

### Tuning

In [ ]:
temps1 = Sys.time()
# le df result tot
RF_tune_AB_tot <- data.frame(mtry = numeric(),
                             ntree = numeric(),
                             maxnodes = numeric(),
                             rmse = numeric(),
                             r_squared = numeric(),
                             mae = numeric())

# Utilisation lapply pour appliquer ForetAlea à chaque combinaison
results_list <- lapply(1:nrow(RF_df_grid), function(i) {
  mtry <- RF_df_grid[i, "mtry"]
  ntree <- RF_df_grid[i, "ntree"]
  maxnodes <- RF_df_grid[i, "nodesize"]

  res <- ForetAlea(var_rep, df_app, df_valid,
                   mtry = mtry, ntree = ntree, maxnodes = maxnodes)

  res_df = data.frame(mtry = mtry,
             ntree = ntree,
             maxnodes = maxnodes,
             rmse = res$RMSE,
             r_squared = res$R_squared,
             mae = res$MAE)
  cat(i,"/",nrow(RF_df_grid),"; ")

  res_df
})

# les résultats en un seul data frame
RF_tune_AB_tot <- do.call(rbind, results_list)

temps2 = Sys.time()
duree = difftime(temps2,temps1)
duree

write.csv2(x =RF_tune_AB_tot,file = paste0("/content/", var_rep, "_RF_tuning.csv"), row.names = FALSE)


## BM_tot

### Import data

In [29]:
# Data
var_rep = "BM_tot"

df_app = read.csv2("/content/BM_tot_train.csv")
df_valid = read.csv2("/content/BM_tot_test.csv")

data = rbind(df_app,df_valid)

dim(data)

[1] 1654   26

### Data preparation

In [30]:
# str(df_app)
# str(df_valid)

ind_var_rep <- which(names(df_app) == var_rep)
trainingtarget <- df_app[, ind_var_rep]
trainingtarget = sqrt(trainingtarget)
training = df_app[,-ind_var_rep]
training[,1:19] = scale(training [,1:19])
df_app = cbind(trainingtarget,training)
colnames(df_app)[ind_var_rep] <- var_rep
df_app = as.data.frame(df_app)

ind_var_rep <- which(names(df_valid) == var_rep)
testtarget <- df_valid[, ind_var_rep]
testtarget <- sqrt(testtarget)
test = df_valid[,-ind_var_rep]
test[,1:19] = scale(test[,1:19])
df_valid = cbind(testtarget,test)
colnames(df_valid)[ind_var_rep] <- var_rep
df_valid = as.data.frame(df_valid)

# str(df_app)
# str(df_valid)

df_app = as.data.frame(df_app)
df_valid = as.data.frame(df_valid)

dim(df_app)
dim(df_valid)

[1] 1212   26

[1] 442  26

### Tuning

In [31]:
temps1 = Sys.time()
# le df result tot
RF_tune_BM_tot <- data.frame(mtry = numeric(),
                             ntree = numeric(),
                             maxnodes = numeric(),
                             rmse = numeric(),
                             r_squared = numeric(),
                             mae = numeric())

# Utilisation lapply pour appliquer ForetAlea à chaque combinaison
results_list <- lapply(1:nrow(RF_df_grid), function(i) {
  mtry <- RF_df_grid[i, "mtry"]
  ntree <- RF_df_grid[i, "ntree"]
  maxnodes <- RF_df_grid[i, "nodesize"]

  res <- ForetAlea(var_rep, df_app, df_valid,
                   mtry = mtry, ntree = ntree, maxnodes = maxnodes)

  res_df = data.frame(mtry = mtry,
                      ntree = ntree,
                      maxnodes = maxnodes,
                      rmse = res$RMSE,
                      r_squared = res$R_squared,
                      mae = res$MAE)
  cat(i,"/",nrow(RF_df_grid))

  res_df
})

# les résultats en un seul data frame
RF_tune_BM_tot <- do.call(rbind, results_list)

temps2 = Sys.time()
duree = difftime(temps2,temps1)
duree

write.csv2(x =RF_tune_BM_tot,file = paste0("/content/", var_rep, "_RF_tuning.csv"), row.names = FALSE)

1 / 9602 / 9603 / 9604 / 9605 / 9606 / 9607 / 9608 / 9609 / 96010 / 96011 / 96012 / 96013 / 96014 / 96015 / 96016 / 96017 / 96018 / 96019 / 96020 / 96021 / 96022 / 96023 / 96024 / 96025 / 96026 / 96027 / 96028 / 96029 / 96030 / 96031 / 96032 / 96033 / 96034 / 96035 / 96036 / 96037 / 96038 / 96039 / 96040 / 96041 / 96042 / 96043 / 96044 / 96045 / 96046 / 96047 / 96048 / 96049 / 96050 / 96051 / 96052 / 96053 / 96054 / 96055 / 96056 / 96057 / 96058 / 96059 / 96060 / 96061 / 96062 / 96063 / 96064 / 96065 / 96066 / 96067 / 96068 / 96069 / 96070 / 96071 / 96072 / 96073 / 96074 / 96075 / 96076 / 96077 / 96078 / 96079 / 96080 / 96081 / 96082 / 96083 / 96084 / 96085 / 96086 / 96087 / 96088 / 96089 / 96090 / 96091 / 96092 / 96093 / 96094 / 96095 / 96096 / 96097 / 96098 / 96099 / 960100 / 960101 / 960102 / 960103 / 960104 / 960105 / 960106 / 960107 / 960108 / 960109 / 960110 / 960111 / 960112 / 960113 / 960114 / 960115 / 960116 / 960117 / 960118 / 960119 / 960120 / 960121 / 960122 / 960123 / 9601

Time difference of 1.246268 hours

## Richesse_tot

### Import data

In [32]:
# Data
var_rep = "Richesse_tot"

df_app = read.csv2("/content/Richesse_tot_train.csv")
df_valid = read.csv2("/content/Richesse_tot_test.csv")

data = rbind(df_app,df_valid)

dim(data)

[1] 3268   26

### Data preparation

In [33]:
# str(df_app)
# str(df_valid)

ind_var_rep <- which(names(df_app) == var_rep)
trainingtarget <- df_app[, ind_var_rep]
#trainingtarget = sqrt(trainingtarget)
training = df_app[,-ind_var_rep]
training[,1:19] = scale(training [,1:19])
df_app = cbind(trainingtarget,training)
colnames(df_app)[ind_var_rep] <- var_rep
df_app = as.data.frame(df_app)

ind_var_rep <- which(names(df_valid) == var_rep)
testtarget <- df_valid[, ind_var_rep]
#testtarget <- sqrt(testtarget)
test = df_valid[,-ind_var_rep]
test[,1:19] = scale(test[,1:19])
df_valid = cbind(testtarget,test)
colnames(df_valid)[ind_var_rep] <- var_rep
df_valid = as.data.frame(df_valid)

# str(df_app)
# str(df_valid)

df_app = as.data.frame(df_app)
df_valid = as.data.frame(df_valid)

dim(df_app)
dim(df_valid)

[1] 2390   26

[1] 878  26

### Tunning

In [34]:

temps1 = Sys.time()
# le df result tot
RF_tune_Richesse_tot <- data.frame(mtry = numeric(),
                             ntree = numeric(),
                             maxnodes = numeric(),
                             rmse = numeric(),
                             r_squared = numeric(),
                             mae = numeric())

# Utilisation lapply pour appliquer ForetAlea à chaque combinaison
results_list <- lapply(1:nrow(RF_df_grid), function(i) {
  mtry <- RF_df_grid[i, "mtry"]
  ntree <- RF_df_grid[i, "ntree"]
  maxnodes <- RF_df_grid[i, "nodesize"]

  res <- ForetAlea(var_rep, df_app, df_valid,
                   mtry = mtry, ntree = ntree, maxnodes = maxnodes)

  res_df = data.frame(mtry = mtry,
                      ntree = ntree,
                      maxnodes = maxnodes,
                      rmse = res$RMSE,
                      r_squared = res$R_squared,
                      mae = res$MAE)
  cat(i,"/",nrow(RF_df_grid))

  res_df
})

# les résultats en un seul data frame
RF_tune_Richesse_tot <- do.call(rbind, results_list)

temps2 = Sys.time()
duree = difftime(temps2,temps1)
duree


write.csv2(x =RF_tune_Richesse_tot,file = paste0("/content/", var_rep, "_RF_tuning.csv"), row.names = FALSE)

1 / 9602 / 9603 / 9604 / 9605 / 9606 / 9607 / 9608 / 9609 / 96010 / 96011 / 96012 / 96013 / 96014 / 96015 / 96016 / 96017 / 96018 / 96019 / 96020 / 96021 / 96022 / 96023 / 96024 / 96025 / 96026 / 96027 / 96028 / 96029 / 96030 / 96031 / 96032 / 96033 / 96034 / 96035 / 96036 / 96037 / 96038 / 96039 / 96040 / 96041 / 96042 / 96043 / 96044 / 96045 / 96046 / 96047 / 96048 / 96049 / 96050 / 96051 / 96052 / 96053 / 96054 / 96055 / 96056 / 96057 / 96058 / 96059 / 96060 / 96061 / 96062 / 96063 / 96064 / 96065 / 96066 / 96067 / 96068 / 96069 / 96070 / 96071 / 96072 / 96073 / 96074 / 96075 / 96076 / 96077 / 96078 / 96079 / 96080 / 96081 / 96082 / 96083 / 96084 / 96085 / 96086 / 96087 / 96088 / 96089 / 96090 / 96091 / 96092 / 96093 / 96094 / 96095 / 96096 / 96097 / 96098 / 96099 / 960100 / 960101 / 960102 / 960103 / 960104 / 960105 / 960106 / 960107 / 960108 / 960109 / 960110 / 960111 / 960112 / 960113 / 960114 / 960115 / 960116 / 960117 / 960118 / 960119 / 960120 / 960121 / 960122 / 960123 / 9601

Time difference of 2.488603 hours

## GBM
Le tunning de GBM a été fait en local: voir script **GBM_tuning.R**